## UCSD Books Dataset Download and Pre-processing

### Downloading UCSD Books Dataset

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1LXpK1UfqtP89H1tYy0pBGHjYk8IhigUK' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1LXpK1UfqtP89H1tYy0pBGHjYk8IhigUK" -O goodreads_books.json.gz && rm -rf /tmp/cookies.txt

### Downloading UCSD Books-Genres Mapping Dataset

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1ah0_KpUterVi-AHxJ03iKD6O0NfbK0md' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1ah0_KpUterVi-AHxJ03iKD6O0NfbK0md" -O gooreads_book_genres_initial.json.gz && rm -rf /tmp/cookies.txt

### Pre-processing

In [0]:
!gunzip goodreads_books.json.gz

In [0]:
import pandas as pd

### Taking a chunk of 250,000 books

In [0]:
df=pd.read_json("goodreads_books.json",lines=True,chunksize=250000)

In [85]:
df

In [4]:
cnt=0
ll=[]
for chunk in df:
  print(chunk["isbn"])
  cnt+=1
  if cnt==1:
    break

0         0312853122
1         0743509986
2                   
3         0743294297
4         0850308712
             ...    
249995    1591886333
249996    0752893769
249997    1595583351
249998              
249999    0688156010
Name: isbn, Length: 250000, dtype: object


In [0]:
ll2=[]
ll2.append(chunk)

In [0]:
ll2[0].to_csv("books.csv")

In [0]:
df3=ll2[0]

In [38]:
df3.head()

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,similar_books,description,format,link,authors,publisher,num_pages,publication_day,isbn13,publication_month,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series
0,0312853122,1,[],US,,"[{'count': '3', 'name': 'to-read'}, {'count': ...",,false,4.00,,[],,Paperback,https://www.goodreads.com/book/show/5333265-w-...,"[{'author_id': '604031', 'role': ''}]",St. Martin's Press,256,1,9780312853129,9,,1984,https://www.goodreads.com/book/show/5333265-w-...,https://images.gr-assets.com/books/1310220028m...,5333265,3,5400751,W.C. Fields: A Life on Film,W.C. Fields: A Life on Film
1,0743509986,6,[],US,,"[{'count': '2634', 'name': 'to-read'}, {'count...",,false,3.23,B000FC0PBC,"[8709549, 17074050, 28937, 158816, 228563, 112...","Anita Diamant's international bestseller ""The ...",Audio CD,https://www.goodreads.com/book/show/1333909.Go...,"[{'author_id': '626222', 'role': ''}]",Simon & Schuster Audio,,1,9780743509985,10,Abridged,2001,https://www.goodreads.com/book/show/1333909.Go...,https://s.gr-assets.com/assets/nophoto/book/11...,1333909,10,1323437,Good Harbor,Good Harbor
2,,7,[189911],US,eng,"[{'count': '58', 'name': 'to-read'}, {'count':...",B00071IKUY,false,4.03,,"[19997, 828466, 1569323, 425389, 1176674, 2627...",Omnibus book club edition containing the Ladie...,Hardcover,https://www.goodreads.com/book/show/7327624-th...,"[{'author_id': '10333', 'role': ''}]","Nelson Doubleday, Inc.",600,,,,Book Club Edition,1987,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,7327624,140,8948723,"The Unschooled Wizard (Sun Wolf and Starhawk, ...","The Unschooled Wizard (Sun Wolf and Starhawk, ..."
3,0743294297,3282,[],US,eng,"[{'count': '7615', 'name': 'to-read'}, {'count...",,false,3.49,B002ENBLOK,"[6604176, 6054190, 2285777, 82641, 7569453, 70...",Addie Downs and Valerie Adler were eight when ...,Hardcover,https://www.goodreads.com/book/show/6066819-be...,"[{'author_id': '9212', 'role': ''}]",Atria Books,368,14,9780743294294,7,,2009,https://www.goodreads.com/book/show/6066819-be...,https://s.gr-assets.com/assets/nophoto/book/11...,6066819,51184,6243154,Best Friends Forever,Best Friends Forever
4,0850308712,5,[],US,,"[{'count': '32', 'name': 'to-read'}, {'count':...",,false,3.40,,[],,,https://www.goodreads.com/book/show/287140.Run...,"[{'author_id': '149918', 'role': ''}]",,,,9780850308716,,,,https://www.goodreads.com/book/show/287140.Run...,https://images.gr-assets.com/books/1413219371m...,287140,15,278577,Runic Astrology: Starcraft and Timekeeping in ...,Runic Astrology: Starcraft and Timekeeping in ...


### Removing books with no description

In [0]:
df3=df3[df3.description!=""]

In [62]:
len(df3)

206606

In [63]:
cnt=list(df3["ratings_count"])
c=[]
for i in cnt:
  try:
    c.append(int(i))
  except:
    print(cnt.index(i))
df3["ratings_count"]=c

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [64]:
len(df3)

206606

### Keeping only English books

In [0]:
df3=df3[df3.language_code=="eng"]

In [67]:
len(df3)

69033

### Keeping the books with number of ratings >=50

In [0]:
df3=df3[df3.ratings_count>=50]

In [69]:
len(df3)

29257

In [70]:
df3.head()

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,similar_books,description,format,link,authors,publisher,num_pages,publication_day,isbn13,publication_month,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series
2,,7,[189911],US,eng,"[{'count': '58', 'name': 'to-read'}, {'count':...",B00071IKUY,false,4.03,,"[19997, 828466, 1569323, 425389, 1176674, 2627...",Omnibus book club edition containing the Ladie...,Hardcover,https://www.goodreads.com/book/show/7327624-th...,"[{'author_id': '10333', 'role': ''}]","Nelson Doubleday, Inc.",600,,,,Book Club Edition,1987,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,7327624,140,8948723,"The Unschooled Wizard (Sun Wolf and Starhawk, ...","The Unschooled Wizard (Sun Wolf and Starhawk, ..."
3,0743294297,3282,[],US,eng,"[{'count': '7615', 'name': 'to-read'}, {'count...",,false,3.49,B002ENBLOK,"[6604176, 6054190, 2285777, 82641, 7569453, 70...",Addie Downs and Valerie Adler were eight when ...,Hardcover,https://www.goodreads.com/book/show/6066819-be...,"[{'author_id': '9212', 'role': ''}]",Atria Books,368,14,9780743294294,7,,2009,https://www.goodreads.com/book/show/6066819-be...,https://s.gr-assets.com/assets/nophoto/book/11...,6066819,51184,6243154,Best Friends Forever,Best Friends Forever
16,0842379428,566,[],US,eng,"[{'count': '6393', 'name': 'to-read'}, {'count...",,false,4.26,B000FCKCJC,"[45366, 246849, 11096385, 19102466, 761197, 11...",What is Heaven really going to be like? What w...,,https://www.goodreads.com/book/show/89376.Heaven,"[{'author_id': '4862', 'role': ''}]",,533,,9780842379427,,,,https://www.goodreads.com/book/show/89376.Heaven,https://images.gr-assets.com/books/1406508230m...,89376,7345,86257,Heaven,Heaven
18,0590417010,193,[],US,eng,"[{'count': '450', 'name': 'to-read'}, {'count'...",,false,4.43,B017RORXNI,"[834493, 452189, 140185, 1897316, 2189812, 424...",In Newbery Medalist Cynthia Rylant's classic b...,Hardcover,https://www.goodreads.com/book/show/89378.Dog_...,"[{'author_id': '5411', 'role': ''}]",Blue Sky Press,40,1,9780590417013,9,,1995,https://www.goodreads.com/book/show/89378.Dog_...,https://images.gr-assets.com/books/1360057676m...,89378,1331,86259,Dog Heaven,Dog Heaven
31,,23,[953679],US,eng,"[{'count': '90', 'name': 'to-read'}, {'count':...",B01ALOWJN0,true,3.49,B01ALOWJN0,[],"BATHS, BANKS AND ROMAN INSURRECTION\nDetective...",,https://www.goodreads.com/book/show/29074697-t...,"[{'author_id': '15104629', 'role': ''}, {'auth...",Amazon Digital Services,,,,,,,https://www.goodreads.com/book/show/29074697-t...,https://s.gr-assets.com/assets/nophoto/book/11...,29074697,192,49305010,The Slaughtered Virgin of Zenopolis (Inspector...,The Slaughtered Virgin of Zenopolis (Inspector...


In [0]:
url = df3["image_url"]

In [72]:
cnt=0
for i in url:
  if "image" in i:
    cnt+=1
print(cnt)

19682


### Books with cover images

In [0]:
df3=df3[df3.image_url.str.contains("images") ]

In [74]:
len(df3)

19682

In [77]:
df3.head()

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,similar_books,description,format,link,authors,publisher,num_pages,publication_day,isbn13,publication_month,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series
2,,7,[189911],US,eng,"[{'count': '58', 'name': 'to-read'}, {'count':...",B00071IKUY,false,4.03,,"[19997, 828466, 1569323, 425389, 1176674, 2627...",Omnibus book club edition containing the Ladie...,Hardcover,https://www.goodreads.com/book/show/7327624-th...,"[{'author_id': '10333', 'role': ''}]","Nelson Doubleday, Inc.",600,,,,Book Club Edition,1987,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,7327624,140,8948723,"The Unschooled Wizard (Sun Wolf and Starhawk, ...","The Unschooled Wizard (Sun Wolf and Starhawk, ..."
16,0842379428,566,[],US,eng,"[{'count': '6393', 'name': 'to-read'}, {'count...",,false,4.26,B000FCKCJC,"[45366, 246849, 11096385, 19102466, 761197, 11...",What is Heaven really going to be like? What w...,,https://www.goodreads.com/book/show/89376.Heaven,"[{'author_id': '4862', 'role': ''}]",,533,,9780842379427,,,,https://www.goodreads.com/book/show/89376.Heaven,https://images.gr-assets.com/books/1406508230m...,89376,7345,86257,Heaven,Heaven
18,0590417010,193,[],US,eng,"[{'count': '450', 'name': 'to-read'}, {'count'...",,false,4.43,B017RORXNI,"[834493, 452189, 140185, 1897316, 2189812, 424...",In Newbery Medalist Cynthia Rylant's classic b...,Hardcover,https://www.goodreads.com/book/show/89378.Dog_...,"[{'author_id': '5411', 'role': ''}]",Blue Sky Press,40,1,9780590417013,9,,1995,https://www.goodreads.com/book/show/89378.Dog_...,https://images.gr-assets.com/books/1360057676m...,89378,1331,86259,Dog Heaven,Dog Heaven
48,,15,[],US,eng,"[{'count': '1492', 'name': 'to-read'}, {'count...",B013Q70BQG,true,3.98,B013Q70BQG,"[386097, 7774253, 576667, 471907, 43174, 38977...",You've Got Mailmeets Julie & Juliain the new f...,,https://www.goodreads.com/book/show/27419760-w...,"[{'author_id': '47231', 'role': ''}]",,,,,,,,https://www.goodreads.com/book/show/27419760-w...,https://images.gr-assets.com/books/1457306424m...,27419760,167,46003673,Wedding Girl,Wedding Girl
84,,5,"[246830, 362583, 362581, 623032]",US,eng,"[{'count': '493', 'name': 'to-read'}, {'count'...",,false,4.28,,"[13590139, 105963, 207585, 10503130, 4645370, ...",The questions plaguing Captain America's dream...,Hardcover,https://www.goodreads.com/book/show/13571772-c...,"[{'author_id': '37450', 'role': ''}]",Hachette Partworks Ltd.,146,,,,,2012,https://www.goodreads.com/book/show/13571772-c...,https://images.gr-assets.com/books/1333287305m...,13571772,51,102217,Captain America: Winter Soldier (The Ultimate ...,Captain America: Winter Soldier (The Ultimate ...


In [0]:
df3=df3.reset_index(drop=True)

In [0]:
import pandas as pd


In [0]:
!cp books.csv /content/drive/My\ Drive

In [0]:
df3=ll2[0]

In [0]:
df3.to_csv("books.csv")

In [0]:
df3=pd.read_csv("books.csv")

In [0]:
!gunzip gooreads_book_genres_initial.json.gz

In [0]:
import pandas as pd
import json
with open('gooreads_book_genres_initial.json') as json_file:      
    data2 = json_file.readlines()
    data2 = list(map(json.loads, data2)) 

df2=pd.DataFrame(data2)

In [0]:
id=df2["book_id"]
id2=[]
for i in id:
  id2.append(str(i))
df2["book_id"]=id2

In [0]:
id=df3["book_id"]
id2=[]
for i in id:
  id2.append(str(i))
df3["book_id"]=id2

### Joining Books with Genres Dataset using Book ID

In [0]:
df_outer = pd.merge(df3, df2, on='book_id', how='left')


In [86]:
len(df_outer)

19682

In [0]:
df_outer.to_csv("booksgenre.csv")

In [88]:
df_outer

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,similar_books,description,format,link,authors,publisher,num_pages,publication_day,isbn13,publication_month,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series,genres
0,,7,[189911],US,eng,"[{'count': '58', 'name': 'to-read'}, {'count':...",B00071IKUY,false,4.03,,"[19997, 828466, 1569323, 425389, 1176674, 2627...",Omnibus book club edition containing the Ladie...,Hardcover,https://www.goodreads.com/book/show/7327624-th...,"[{'author_id': '10333', 'role': ''}]","Nelson Doubleday, Inc.",600,,,,Book Club Edition,1987,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,7327624,140,8948723,"The Unschooled Wizard (Sun Wolf and Starhawk, ...","The Unschooled Wizard (Sun Wolf and Starhawk, ...","{'fantasy, paranormal': 31, 'fiction': 8, 'mys..."
1,0842379428,566,[],US,eng,"[{'count': '6393', 'name': 'to-read'}, {'count...",,false,4.26,B000FCKCJC,"[45366, 246849, 11096385, 19102466, 761197, 11...",What is Heaven really going to be like? What w...,,https://www.goodreads.com/book/show/89376.Heaven,"[{'author_id': '4862', 'role': ''}]",,533,,9780842379427,,,,https://www.goodreads.com/book/show/89376.Heaven,https://images.gr-assets.com/books/1406508230m...,89376,7345,86257,Heaven,Heaven,{'non-fiction': 163}
2,0590417010,193,[],US,eng,"[{'count': '450', 'name': 'to-read'}, {'count'...",,false,4.43,B017RORXNI,"[834493, 452189, 140185, 1897316, 2189812, 424...",In Newbery Medalist Cynthia Rylant's classic b...,Hardcover,https://www.goodreads.com/book/show/89378.Dog_...,"[{'author_id': '5411', 'role': ''}]",Blue Sky Press,40,1,9780590417013,9,,1995,https://www.goodreads.com/book/show/89378.Dog_...,https://images.gr-assets.com/books/1360057676m...,89378,1331,86259,Dog Heaven,Dog Heaven,"{'children': 109, 'fiction': 13, 'non-fiction'..."
3,,15,[],US,eng,"[{'count': '1492', 'name': 'to-read'}, {'count...",B013Q70BQG,true,3.98,B013Q70BQG,"[386097, 7774253, 576667, 471907, 43174, 38977...",You've Got Mailmeets Julie & Juliain the new f...,,https://www.goodreads.com/book/show/27419760-w...,"[{'author_id': '47231', 'role': ''}]",,,,,,,,https://www.goodreads.com/book/show/27419760-w...,https://images.gr-assets.com/books/1457306424m...,27419760,167,46003673,Wedding Girl,Wedding Girl,"{'romance': 27, 'fiction': 22}"
4,,5,"[246830, 362583, 362581, 623032]",US,eng,"[{'count': '493', 'name': 'to-read'}, {'count'...",,false,4.28,,"[13590139, 105963, 207585, 10503130, 4645370, ...",The questions plaguing Captain America's dream...,Hardcover,https://www.goodreads.com/book/show/13571772-c...,"[{'author_id': '37450', 'role': ''}]",Hachette Partworks Ltd.,146,,,,,2012,https://www.goodreads.com/book/show/13571772-c...,https://images.gr-assets.com/books/1333287305m...,13571772,51,102217,Captain America: Winter Soldier (The Ultimate ...,Captain America: Winter Soldier (The Ultimate ...,"{'comics, graphic': 433, 'fiction': 19, 'fanta..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19677,1452857237,30,[253252],US,eng,"[{'count': '241', 'name': 'to-read'}, {'count'...",,false,3.99,B003L77PZM,"[2098363, 12480650, 935472, 12698687, 1441971,...",After winning back the money his uncle gambled...,Paperback,https://www.goodreads.com/book/show/8279428-th...,"[{'author_id': '155650', 'role': ''}]",CreateSpace,214,7,9781452857237,5,,2010,https://www.goodreads.com/book/show/8279428-th...,https://images.gr-assets.com/books/1330591019m...,8279428,431,13127736,"The Wrong Husband (Nebraska Historicals, #5)","The Wrong Husband (Nebraska Historicals, #5)","{'romance': 45, 'history, historical fiction, ..."
19678,0747594848,78,[214957],US,eng,"[{'count': '1415', 'name': 'to-read'}, {'count...",,false,4.27,B003UYUOT0,"[13012638, 4621258, 1246570, 6120544, 13603796...",Otto is on the run from U

In [0]:
import pandas as pd
df_outer=pd.read_csv("booksgenre.csv")

In [9]:
df_outer.iloc[1]["genres"]

{'fiction': 219, 'history, historical fiction, biography': 5}

In [0]:
a=df_outer["genres"]

In [90]:
list(a[0].keys())

['fantasy, paranormal', 'fiction', 'mystery, thriller, crime', 'poetry']

In [0]:
b=[]
for i in a:
  b.extend(list(i.keys()))

In [92]:
len(b)

60910

In [0]:
c=set(b)

### Genre Set

In [94]:
c

{'children',
 'comics, graphic',
 'fantasy, paranormal',
 'fiction',
 'history, historical fiction, biography',
 'mystery, thriller, crime',
 'non-fiction',
 'poetry',
 'romance',
 'young-adult'}

In [0]:
e={1,2,3,4,5,6,7,8,9,10}

In [96]:
new_dict = dict(zip(e, c))
print(new_dict)
print(type(new_dict))

{1: 'non-fiction', 2: 'mystery, thriller, crime', 3: 'fantasy, paranormal', 4: 'young-adult', 5: 'comics, graphic', 6: 'history, historical fiction, biography', 7: 'fiction', 8: 'children', 9: 'romance', 10: 'poetry'}
<class 'dict'>


In [0]:
dfk=df_outer[["book_id","genres"]]

In [98]:
dfk

,book_id,genres
0,7327624,"{'fantasy, paranormal': 31, 'fiction': 8, 'mys..."
1,89376,{'non-fiction': 163}
2,89378,"{'children': 109, 'fiction': 13, 'non-fiction'..."
3,27419760,"{'romance': 27, 'fiction': 22}"
4,13571772,"{'comics, graphic': 433, 'fiction': 19, 'fanta..."
...,...,...
19677,8279428,"{'romance': 45, 'history, historical fiction, ..."
19678,6275767,"{'fiction': 58, 'young-adult': 36, 'mystery, t..."
19679,24820339,"{'fiction': 51, 'non-fiction': 25, 'history, h..."
19680,12960440,"{'fantasy, paranormal': 1, 'fiction': 2}"


In [0]:
lll=df2["genres"]

In [100]:
cnt=0
for i in lll:
  if i=={}:
    cnt+=1
print(cnt)

409513


In [101]:
len(lll)

2360655

In [0]:
def get_key(val): 
    for key, value in new_dict.items(): 
         if val == value: 
             return key

In [0]:
genres=[]
for i in range(len(dfk)):
  for j in list(dfk.iloc[i]["genres"].keys()):
    genres.append([dfk.iloc[i]["book_id"],get_key(j)])


In [0]:
import pickle
f=open("genremap.pkl","wb")
pickle.dump(genres,f)

In [105]:
genres[2]

['7327624', 2]

In [0]:
books_genre=pd.DataFrame(genres)

In [107]:
len(books_genre)

60910

In [0]:
books_genre.columns=["book_id","genre_id"]

In [0]:
books_genre.to_csv("books_genre.csv")

## Created Book_ID to Genre_ID Mapping

In [110]:
books_genre

,book_id,genre_id
0,7327624,3
1,7327624,7
2,7327624,2
3,7327624,10
4,89376,1
...,...,...
60905,12960440,7
60906,27160030,9
60907,27160030,6
60908,27160030,7


In [0]:
genre_map=pd.DataFrame(new_dict.items(),columns=["id","name"])

In [112]:
genre_map

,id,name
0,1,non-fiction
1,2,"mystery, thriller, crime"
2,3,"fantasy, paranormal"
3,4,young-adult
4,5,"comics, graphic"
5,6,"history, historical fiction, biography"
6,7,fiction
7,8,children
8,9,romance
9,10,poetry


In [0]:
genre_map.to_csv("genre_map.csv")

In [0]:
https://drive.google.com/uc?id=19cdwyXwfXx_HDIgxXaHzH0mrx8nMyLvC

### Downloading UCSD Author Dataset

In [114]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=19cdwyXwfXx_HDIgxXaHzH0mrx8nMyLvC' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=19cdwyXwfXx_HDIgxXaHzH0mrx8nMyLvC" -O goodreads_book_authors.json.gz && rm -rf /tmp/cookies.txt

--2020-04-20 10:03:11--  https://docs.google.com/uc?export=download&confirm=&id=19cdwyXwfXx_HDIgxXaHzH0mrx8nMyLvC
Resolving docs.google.com (docs.google.com)... 64.233.170.113, 64.233.170.138, 64.233.170.139, ...
Connecting to docs.google.com (docs.google.com)|64.233.170.113|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-14-50-docs.googleusercontent.com/docs/securesc/3l8t91m0ngb72srmutus63kbhgdtqvff/1b3udurk0nugjiakk4speckvbirt6k02/1587376950000/02073811625855954957/17440007148545228807Z/19cdwyXwfXx_HDIgxXaHzH0mrx8nMyLvC?e=download [following]
--2020-04-20 10:03:13--  https://doc-14-50-docs.googleusercontent.com/docs/securesc/3l8t91m0ngb72srmutus63kbhgdtqvff/1b3udurk0nugjiakk4speckvbirt6k02/1587376950000/02073811625855954957/17440007148545228807Z/19cdwyXwfXx_HDIgxXaHzH0mrx8nMyLvC?e=download
Resolving doc-14-50-docs.googleusercontent.com (doc-14-50-docs.googleusercontent.com)... 172.217.203.132, 2607:f8b0:400c:c07::84
Connecting to

In [0]:
!gunzip goodreads_book_authors.json.gz

In [0]:
import pandas as pd
import json
with open('goodreads_book_authors.json') as json_file:      
    data2 = json_file.readlines()
    data2 = list(map(json.loads, data2)) 

df4=pd.DataFrame(data2)

In [117]:
df3.head()

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,similar_books,description,format,link,authors,publisher,num_pages,publication_day,isbn13,publication_month,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series
0,,7,[189911],US,eng,"[{'count': '58', 'name': 'to-read'}, {'count':...",B00071IKUY,false,4.03,,"[19997, 828466, 1569323, 425389, 1176674, 2627...",Omnibus book club edition containing the Ladie...,Hardcover,https://www.goodreads.com/book/show/7327624-th...,"[{'author_id': '10333', 'role': ''}]","Nelson Doubleday, Inc.",600,,,,Book Club Edition,1987,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,7327624,140,8948723,"The Unschooled Wizard (Sun Wolf and Starhawk, ...","The Unschooled Wizard (Sun Wolf and Starhawk, ..."
1,0842379428,566,[],US,eng,"[{'count': '6393', 'name': 'to-read'}, {'count...",,false,4.26,B000FCKCJC,"[45366, 246849, 11096385, 19102466, 761197, 11...",What is Heaven really going to be like? What w...,,https://www.goodreads.com/book/show/89376.Heaven,"[{'author_id': '4862', 'role': ''}]",,533,,9780842379427,,,,https://www.goodreads.com/book/show/89376.Heaven,https://images.gr-assets.com/books/1406508230m...,89376,7345,86257,Heaven,Heaven
2,0590417010,193,[],US,eng,"[{'count': '450', 'name': 'to-read'}, {'count'...",,false,4.43,B017RORXNI,"[834493, 452189, 140185, 1897316, 2189812, 424...",In Newbery Medalist Cynthia Rylant's classic b...,Hardcover,https://www.goodreads.com/book/show/89378.Dog_...,"[{'author_id': '5411', 'role': ''}]",Blue Sky Press,40,1,9780590417013,9,,1995,https://www.goodreads.com/book/show/89378.Dog_...,https://images.gr-assets.com/books/1360057676m...,89378,1331,86259,Dog Heaven,Dog Heaven
3,,15,[],US,eng,"[{'count': '1492', 'name': 'to-read'}, {'count...",B013Q70BQG,true,3.98,B013Q70BQG,"[386097, 7774253, 576667, 471907, 43174, 38977...",You've Got Mailmeets Julie & Juliain the new f...,,https://www.goodreads.com/book/show/27419760-w...,"[{'author_id': '47231', 'role': ''}]",,,,,,,,https://www.goodreads.com/book/show/27419760-w...,https://images.gr-assets.com/books/1457306424m...,27419760,167,46003673,Wedding Girl,Wedding Girl
4,,5,"[246830, 362583, 362581, 623032]",US,eng,"[{'count': '493', 'name': 'to-read'}, {'count'...",,false,4.28,,"[13590139, 105963, 207585, 10503130, 4645370, ...",The questions plaguing Captain America's dream...,Hardcover,https://www.goodreads.com/book/show/13571772-c...,"[{'author_id': '37450', 'role': ''}]",Hachette Partworks Ltd.,146,,,,,2012,https://www.goodreads.com/book/show/13571772-c...,https://images.gr-assets.com/books/1333287305m...,13571772,51,102217,Captain America: Winter Soldier (The Ultimate ...,Captain America: Winter Soldier (The Ultimate ...


In [118]:
df4.head()

,average_rating,author_id,text_reviews_count,name,ratings_count
0,3.98,604031,7,Ronald J. Fields,49
1,4.08,626222,28716,Anita Diamant,546796
2,3.92,10333,5075,Barbara Hambly,122118
3,3.68,9212,36262,Jennifer Weiner,888522
4,3.82,149918,96,Nigel Pennick,1740


In [119]:
len(df4)

829529

In [0]:
author_id=list(df4["author_id"])
author_name=list(df4["name"])

In [0]:
author=list(df3["authors"])

In [122]:
author[4]

[{'author_id': '37450', 'role': ''}]

In [0]:
author1=[]
name=[]
for i in author:
  n=[]
  for j in i:
    for k in j.keys():
      if k=='author_id':
        l=author_id.index(j[k])
        m=author_name[l]
        n.append(m)
  name.append(n)

In [124]:
len(name)

19682

In [0]:
ind=[]
for i in name:
  if len(i)>1:
    ind.append(name.index(i))

In [126]:
name[9]

['Douglas Preston', 'Lincoln Child']

## Changing the Author column dictionaries to Author names

In [0]:
df3["authors"]=name

In [128]:
df3.head()

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,similar_books,description,format,link,authors,publisher,num_pages,publication_day,isbn13,publication_month,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series
0,,7,[189911],US,eng,"[{'count': '58', 'name': 'to-read'}, {'count':...",B00071IKUY,false,4.03,,"[19997, 828466, 1569323, 425389, 1176674, 2627...",Omnibus book club edition containing the Ladie...,Hardcover,https://www.goodreads.com/book/show/7327624-th...,[Barbara Hambly],"Nelson Doubleday, Inc.",600,,,,Book Club Edition,1987,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,7327624,140,8948723,"The Unschooled Wizard (Sun Wolf and Starhawk, ...","The Unschooled Wizard (Sun Wolf and Starhawk, ..."
1,0842379428,566,[],US,eng,"[{'count': '6393', 'name': 'to-read'}, {'count...",,false,4.26,B000FCKCJC,"[45366, 246849, 11096385, 19102466, 761197, 11...",What is Heaven really going to be like? What w...,,https://www.goodreads.com/book/show/89376.Heaven,[Randy Alcorn],,533,,9780842379427,,,,https://www.goodreads.com/book/show/89376.Heaven,https://images.gr-assets.com/books/1406508230m...,89376,7345,86257,Heaven,Heaven
2,0590417010,193,[],US,eng,"[{'count': '450', 'name': 'to-read'}, {'count'...",,false,4.43,B017RORXNI,"[834493, 452189, 140185, 1897316, 2189812, 424...",In Newbery Medalist Cynthia Rylant's classic b...,Hardcover,https://www.goodreads.com/book/show/89378.Dog_...,[Cynthia Rylant],Blue Sky Press,40,1,9780590417013,9,,1995,https://www.goodreads.com/book/show/89378.Dog_...,https://images.gr-assets.com/books/1360057676m...,89378,1331,86259,Dog Heaven,Dog Heaven
3,,15,[],US,eng,"[{'count': '1492', 'name': 'to-read'}, {'count...",B013Q70BQG,true,3.98,B013Q70BQG,"[386097, 7774253, 576667, 471907, 43174, 38977...",You've Got Mailmeets Julie & Juliain the new f...,,https://www.goodreads.com/book/show/27419760-w...,[Stacey Ballis],,,,,,,,https://www.goodreads.com/book/show/27419760-w...,https://images.gr-assets.com/books/1457306424m...,27419760,167,46003673,Wedding Girl,Wedding Girl
4,,5,"[246830, 362583, 362581, 623032]",US,eng,"[{'count': '493', 'name': 'to-read'}, {'count'...",,false,4.28,,"[13590139, 105963, 207585, 10503130, 4645370, ...",The questions plaguing Captain America's dream...,Hardcover,https://www.goodreads.com/book/show/13571772-c...,[Ed Brubaker],Hachette Partworks Ltd.,146,,,,,2012,https://www.goodreads.com/book/show/13571772-c...,https://images.gr-assets.com/books/1333287305m...,13571772,51,102217,Captain America: Winter Soldier (The Ultimate ...,Captain America: Winter Soldier (The Ultimate ...


In [0]:
df3.to_csv("books.csv")

In [0]:
df_outer["authors"]=name

In [0]:
df_outer.to_csv("booksgenre.csv")

In [133]:
df_outer.head()

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,similar_books,description,format,link,authors,publisher,num_pages,publication_day,isbn13,publication_month,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series,genres
0,,7,[189911],US,eng,"[{'count': '58', 'name': 'to-read'}, {'count':...",B00071IKUY,false,4.03,,"[19997, 828466, 1569323, 425389, 1176674, 2627...",Omnibus book club edition containing the Ladie...,Hardcover,https://www.goodreads.com/book/show/7327624-th...,[Barbara Hambly],"Nelson Doubleday, Inc.",600,,,,Book Club Edition,1987,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,7327624,140,8948723,"The Unschooled Wizard (Sun Wolf and Starhawk, ...","The Unschooled Wizard (Sun Wolf and Starhawk, ...","{'fantasy, paranormal': 31, 'fiction': 8, 'mys..."
1,0842379428,566,[],US,eng,"[{'count': '6393', 'name': 'to-read'}, {'count...",,false,4.26,B000FCKCJC,"[45366, 246849, 11096385, 19102466, 761197, 11...",What is Heaven really going to be like? What w...,,https://www.goodreads.com/book/show/89376.Heaven,[Randy Alcorn],,533,,9780842379427,,,,https://www.goodreads.com/book/show/89376.Heaven,https://images.gr-assets.com/books/1406508230m...,89376,7345,86257,Heaven,Heaven,{'non-fiction': 163}
2,0590417010,193,[],US,eng,"[{'count': '450', 'name': 'to-read'}, {'count'...",,false,4.43,B017RORXNI,"[834493, 452189, 140185, 1897316, 2189812, 424...",In Newbery Medalist Cynthia Rylant's classic b...,Hardcover,https://www.goodreads.com/book/show/89378.Dog_...,[Cynthia Rylant],Blue Sky Press,40,1,9780590417013,9,,1995,https://www.goodreads.com/book/show/89378.Dog_...,https://images.gr-assets.com/books/1360057676m...,89378,1331,86259,Dog Heaven,Dog Heaven,"{'children': 109, 'fiction': 13, 'non-fiction'..."
3,,15,[],US,eng,"[{'count': '1492', 'name': 'to-read'}, {'count...",B013Q70BQG,true,3.98,B013Q70BQG,"[386097, 7774253, 576667, 471907, 43174, 38977...",You've Got Mailmeets Julie & Juliain the new f...,,https://www.goodreads.com/book/show/27419760-w...,[Stacey Ballis],,,,,,,,https://www.goodreads.com/book/show/27419760-w...,https://images.gr-assets.com/books/1457306424m...,27419760,167,46003673,Wedding Girl,Wedding Girl,"{'romance': 27, 'fiction': 22}"
4,,5,"[246830, 362583, 362581, 623032]",US,eng,"[{'count': '493', 'name': 'to-read'}, {'count'...",,false,4.28,,"[13590139, 105963, 207585, 10503130, 4645370, ...",The questions plaguing Captain America's dream...,Hardcover,https://www.goodreads.com/book/show/13571772-c...,[Ed Brubaker],Hachette Partworks Ltd.,146,,,,,2012,https://www.goodreads.com/book/show/13571772-c...,https://images.gr-assets.com/books/1333287305m...,13571772,51,102217,Captain America: Winter Soldier (The Ultimate ...,Captain America: Winter Soldier (The Ultimate ...,"{'comics, graphic': 433, 'fiction': 19, 'fanta..."


In [0]:
!cp booksgenre.csv /content/drive/My\ Drive